# Ungraded Lab Part 2 - Consume a Machine Learning Model

Welcome to part 2 of this ungraded lab! **Before going forward check that the server from part 1 is still running.**

In this notebook you will code a minimal client that uses Python's `requests` library to interact with your server.

In [ ]:
import os
import io
import cv2
import requests
import numpy as np
from IPython.display import Image, display

## Understand the URL


### Breaking down the URL

If you played with fastAPI's client maybe you noticed that the way the requests were made was by pointing to an URL and appending some parameters to it.

Your server is now hosted in the URL [http://localhost:8000/](http://localhost:8000/).

The endpoint that serves your model is the `/predict` endpoint.

And if you stuck with the tiny version of YOLOV3 the model used is `yolov3-tiny`.

In [ ]:
base_url = 'http://localhost:8000'
endpoint = '/predict'
model = 'yolov3-tiny'

By appending the base URL and the endpoint you will get the full URL (without parameters) needed to consume your model.

In [ ]:
url_with_endpoint_no_params = base_url + endpoint
url_with_endpoint_no_params

To set any parameters the common syntax is to add a "?" character followed by the name of the parameter and its value.

Let's do it and see how the final URL would look like:

In [ ]:
full_url = url_with_endpoint_no_params + "?model=" + model
full_url

You might notice that this endpoint expects both a model's name and an image. But since the image is more complex it is not passed within the URL. Instead we can use the `requests` library to handle this.

## Do a request to your server

### Code the response_from_server function

Remember that this endpoint expects a POST HTTP request. To create such request you can use the `post` function from the requests library. 

To pass the file along with the request, create a dictionary indicating the name of the file ('file' in this case) and the actual file.

When doing a request you can check the `status code` of the response it produced. **A status code of 200 means that everything went well.**

In [ ]:
def response_from_server(url, image_file, verbose=True):
    """Makes a POST request to the server and returns the response.

    Args:
        url (str): URL that the request is sent to.
        image_file (_io.BufferedReader): File to upload, should be an image.
        verbose (bool): True if the status of the response should be printed. False otherwise.

    Returns:
        requests.models.Response: Response from the server.
    """
    
    files = {'file': image_file}
    response = requests.post(url, files=files)
    status_code = response.status_code
    if verbose:
        msg = "Everything went well!" if status_code == 200 else "There was an error when handling the request."
        print(msg)
    return response

To test this function you can open a file in your filesystem and pass it as a parameter alongside the URL:

In [ ]:
with open("images/clock2.jpg", "rb") as image_file:
    prediction = response_from_server(full_url, image_file)

It is great to know that the request was succesful but you are not getting any information about the objects in the image.

To get the image with the bounding boxes and labels you need to parse the content of the response into an appropiate format. This process looks very similar to how you read the raw image into a cv2 image in the server.

Before doing this, let's create a directory called `images_predicted` to save the image to:

In [ ]:
dir_name = "images_predicted"
if not os.path.exists(dir_name):
    os.mkdir(dir_name)


### Create the display_image_from_response function

In [ ]:
def display_image_from_response(response):
    """Display image within server's response.

    Args:
        response (requests.models.Response): The response from the server after object detection.
    """
    
    image_stream = io.BytesIO(response.content)
    image_stream.seek(0)
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    filename = "image_with_objects.jpeg"
    cv2.imwrite(f'images_predicted/{filename}', image)
    display(Image(f'images_predicted/{filename}'))

In [ ]:
display_image_from_response(prediction)

Now you can consume your object detection model through your own client!

Test it out on some other images:

In [ ]:
image_files = [
    'car2.jpg',
    'clock3.jpg',
    'apples.jpg'
]

for image_file in image_files:
    with open(f"images/{image_file}", "rb") as image_file:
        prediction = response_from_server(full_url, image_file, verbose=False)
    
    display_image_from_response(prediction)

**Congratulations on finishing this ungraded lab!** Real life clients and servers have a lot more going on in terms of security and performance. However, the code you just saw is close to the one on a real production environment. We hope you feel more familiar with the process of deploying a Deep Learning model, consuming it and encourage you to keep studying.

**Keep it up!**